# Turing Machines

This module encodes non-deterministic Turing Machines with a doubly-infinite tape (call it the "dub TM" :-) ). We begin with a tape containing exactly the given input and let the computation begin facing the left-end of the given input string.

The given input string could be empty (epsilon). This situation translates to the TM's initial head facing a sea of blanks on the tape -- both to its left and to its right. No amount of "sniff-check" will let you sniff anything other than a sea (of blanks).

Each attempt to ``fall off either end of the tape'' is met with an allocation of some number (currently 8) of extra blank characters. Halting configurations as well as paths leading to them are maintained. All executions are bounded by a constant __fuel__ that the user can progressively increase till the desired acceptances are seen (or the user surmises nontermination). This is like the gas in your tank! When you run out of gas, you are forced to halt.

We maintain nsteps as remaining "fuel" per thread (non-deterministic) of execution. When a thread runs out of fuel, it stops running. A run ends with a printout of the terminal configurations plus the fuel remaining per thread. This is ideal from the point of explaining nondeterministic runtimes. We elaborate now.

An NDTM accepts a string in nondeterminstic polynomial time (NP) if there is an accepting computational history of polynomial length. In our NDTM simulation, you'll see the remaining fuel per thread. Any thread with the least fuel consumption and still accepts is the one we go by in measuring runtimes!

In [1]:
from jove.SystemImports       import *
from jove.DotBashers          import *
from jove.TransitionSelectors import *

# Turing Machines

## Basic Definitions

Turing Machines are structures 

  $(Q, Sigma, Gamma, Delta, q0, B, F)$
  
where

 * $Q$       : Finite non-empty set of states

 * $Sigma$   : Finite non-empty input alphabet

 * $Gamma$   : Finite non-empty tape alphabet (subsumes Sigma, so that the user-given input can be written on the tape; also, $Gamma$ always includes $B$, representing an empty space (``blank'') on the tape).
 
   - Computations are set up by writing the user-given input on the tape
   
   - The portion of the tape before and after the user input is filled with an infinite **sea of blanks** (we allocate it on demand).
   
       - Note: In our TMs, the blank character 
       is user-selectable. The preferred 
         blank symbol is "." (dot).

 * $Delta$   : A transition function that takes a state, a current tape
 symbol being scanned

$Delta$'s signature is
       
$(Q \times Gamma) \rightarrow P(Q \times Gamma \times \{L,R,S\} )$

This means that the TM, in state q, while scanning tape symbol s will choose one of the (q1, g, Dir) tripes. Here, q1 is the next control state, is the new tape symbol that replaces what's being looked at, and Dir is one of Left, Right, or Same, encoded by "L", "R", or "S". 

A TM is ``stuck'' if it cannot fire __any__ transition from a given configuration. Such terminal configurations are halting configurations with a __reject__ status. All final states are also terminal configurations with an __accept__ status.
 

# Computation wrt Instantaneous Descriptions (ID)

Let us define the transition function (transition table) as follows:

* { (q, g) : { (q1, g1, D1), (q2, g2, D2), ... }

An ID of a TM is, mathematically, a triple

     (q, tape, position-of-tape-being-ogled-at)
 
For our (more practical) TM, the ID is a quadruple

     (q, hi, tape, fuel)
    
where 

- q is the present control state of the TM,

- hi is the head index (into the tape, 
treating index=0 as the leftmost position of the tape).

- tape is the string that constitutes the tape, and

- fuel is the amount of "fuel" (computational steps) left
  for this ID. 
  
    - Note that  when we split IDs, we convey the same
      amount of remaining fuel to the threads being spawned.

 

Examples now follow, and they will drive home these ideas.


 Consistency checker(s) for TM have gone to DotBashers

# Example DTM: accepts "w#w"

This TM is made deterministic by going to exactly one new ID (the set of next states in the transition table has a cardinality of one). We will enter the TM (called wpw) and use dotObj_tm to plot it.

In [2]:
# Example TM that recognizes w#w. We use "." to denote blanks.
# Blanks are of course user-selectable
wpw = {
 "Q": {"q0","q1","q2","q3","q4","q5","q6","q7","q8","q9","q10"},
 "Sigma" : {'0','1'},
 "Gamma" : {'0','1','X','Y','#','.'},
 "Delta" : {  
     ("q0","0") : { ("q1","X","R")},
     ("q0","#") : { ("q5","#","R")},
     ("q0","1") : { ("q7","Y","R")},
     #
     ("q1","0") : { ("q1","0","R")},     
     ("q1","1") : { ("q1","1","R")},
     ("q1","#") : { ("q2","#","R")},
     #
     ("q2","X") : { ("q2","X","R")},
     ("q2","Y") : { ("q2","Y","R")},     
     ("q2","0") : { ("q3","X","L")},
     #
     ("q3","X") : { ("q3","X","L")},  
     ("q3","Y") : { ("q3","Y","L")},
     ("q3","#") : { ("q4","#","L")}, 
     #
     ("q4","0") : { ("q4","0","L")},
     ("q4","1") : { ("q4","1","L")},     
     ("q4","X") : { ("q0","X","R")},
     ("q4","Y") : { ("q0","Y","R")},
     #
     ("q5","X") : { ("q5","X","R")},     
     ("q5","Y") : { ("q5","Y","R")},
     ("q5",".") : { ("q6",".","R")},  
     #
     ("q7","0") : { ("q7","0","R")},
     ("q7","1") : { ("q7","1","R")},     
     ("q7","#") : { ("q8","#","R")},
     #
     ("q8","X") : { ("q8","X","R")},     
     ("q8","Y") : { ("q8","Y","R")},
     ("q8","1") : { ("q9","Y","L")}, 
     #
     ("q9","X") : { ("q9","X","L")},     
     ("q9","Y") : { ("q9","Y","L")},
     ("q9","#") : { ("q10","#","L")},  
     #
     ("q10","0") : { ("q10","0","L")},     
     ("q10","1") : { ("q10","1","L")},
     ("q10","X") : { ("q0","X","R")},
     ("q10","Y") : { ("q0","Y","R")},
 },
 "q0"    : "q0",
 "B"     : '.',
 "F"     : {"q6"}
}

In [3]:
wpw

In [4]:
wpwobj = dotObj_tm(wpw)

In [5]:
wwndtm = {
 "Q": {"q0","q1","q2","q3","q4","q5","q6","q7","q8","q9","q10",
       "q11","q12","q13","q14"},
 "Sigma" : {'0','1'},
 "Gamma" : {'0','1','X','Y','2','3',
            'P','Q','.'},
 "Delta" : {  
     ("q0",".") : { ("q1",".","R")},
     ("q0","0") : { ("q14","0","S")},
     ("q0","1") : { ("q14","1","S")},
     #
     ("q14","0") : { ("q14","0","R"), ('q2','X','L')},
     ("q14","1") : { ("q14","1","R"), ('q2','Y','L')},
     #
     ("q2","0") : { ("q2","0","L")},
     ("q2","1") : { ("q2","1","L")},     
     ("q2",".") : { ("q3",".","R")},
     #
     ("q3","X") : { ("q6","X","R")},  
     ("q3","Y") : { ("q7","Y","R")},
     #
     ("q3","0") : { ("q4","P","R")},
     ("q3","1") : { ("q5","Q","R")},          
     #
     ("q3","2") : { ("q12","2","R")},
     ("q3","3") : { ("q12","3","R")},
     #
     #--
     ("q4","0") : { ("q4","0","R")},
     ("q4","1") : { ("q4","1","R")},
     ("q4","2") : { ("q4","2","R")},
     ("q4","3") : { ("q4","3","R")},
     ("q4","X") : { ("q10","2","R")},     
     #
     ("q5","0") : { ("q5","0","R")},
     ("q5","1") : { ("q5","1","R")},
     ("q5","2") : { ("q5","2","R")},
     ("q5","3") : { ("q5","3","R")},
     ("q5","Y") : { ("q11","3","R")},     
     #     
     ("q10","0") : { ("q8","X","L")},     
     ("q10","1") : { ("q8","Y","L")},
     ("q10",".") : { ("q8",".","L")},
     #     
     ("q11","0") : { ("q9","X","L")},     
     ("q11","1") : { ("q9","Y","L")},
     ("q11",".") : { ("q9",".","L")},     
     #
     ("q8","0") : { ("q8","0","L")},     
     ("q8","1") : { ("q8","1","L")},
     ("q8","2") : { ("q8","2","L")},
     ("q8","3") : { ("q8","3","L")},
     ("q8","P") : { ("q3","P","R")},
     #
     ("q9","0") : { ("q9","0","L")},     
     ("q9","1") : { ("q9","1","L")},
     ("q9","2") : { ("q9","2","L")},
     ("q9","3") : { ("q9","3","L")},
     ("q9","Q") : { ("q3","Q","R")},
     #
     ("q12","2") : { ("q12","2","R")},
     ("q12","3") : { ("q12","3","R")},
     ("q12",".") : { ("q13",".","R")}
 },
 "q0"    : "q0",
 "B"     : '.',
 "F"     : {"q1","q13"}
}


In [6]:
wpwobj.source

In [7]:
 TAPE_ALLOC_SIZE = 8
def step_tm(T, q_hi_tape_fuel, path, haltList):
    """Inputs: * A TM T
               * An ID -- a tuple q_hi_tape_fuel capturing
               - q:  the present state of the TM
               - hi: the head index that is initialized to 0.
               - tape: The full string representing the tape. 
                 tape[0] is the element at the leftmost position.
               - fuel: The amount of fuel left in this thread.
                 When an NDTM splits, it imparts the currently
                 remaining amount of fuel to all the progeny 
                 threads.
               * A path that leads to q_hi_tape_fuel
               * A haltList of halting configurations that 
                 builds up. Each halting config kept with path
                 leading to it.
       Output: A pair (l_id_path, nhaltList)
               * l_id_path is a list  
                  [ ((nq, nhi, ntape, nfuel), npath) ]   
                where (nq, nhi, ntape, nfuel) is the new ID 
                and npath is the extended path reaching it.
               * nhaltList is the extended halting config list.
       Detail: When the head is about to fall off either end, we 
               allocate TAPE_ALLOC_SIZE blank characters, thus keeping
               the head on the tape.
    """ 
    (q, hi, tape, fuel) = q_hi_tape_fuel
    extpath             = path + [q_hi_tape_fuel]
    nl_id_path          = []
    
    if (hi == len(tape)):
        # Going beyond end of allocated tape; allocate more!
        print("Allocating ", TAPE_ALLOC_SIZE, " tape cells to the RIGHT!")
        tape = tape + T["B"]*TAPE_ALLOC_SIZE
        
    if (q, tape[hi]) not in T["Delta"]:
        # No move on (q, tape[hi]), so record halt lset; return
        return (nl_id_path, haltList + [(q_hi_tape_fuel, path)])
               
    l_nq_ng_dirn      = T["Delta"][(q, tape[hi])] 
    
    for nq_ng_dirn in l_nq_ng_dirn:
        (nq, ng, dirn) = nq_ng_dirn
        # Head attempts to move to the left of the left-end
        if (hi==0) and (dirn=="L"):
            print("Allocating ", TAPE_ALLOC_SIZE, " tape cells to the LEFT!")
            ntape = T["B"]*TAPE_ALLOC_SIZE + ng + tape[1:]  
            nhi   = TAPE_ALLOC_SIZE - 1  # Do the left move too!
        else:
            ntape = tape[0:hi] +  ng  + tape[hi+1:len(tape)]
            nhi = (hi+1 if dirn=="R"
                   else ((hi-1) if dirn=="L" 
                         else (hi if dirn=="S"
                              else print("Illegal direction!"))))
           
        if (fuel > 0):
            nl_id_path += [((nq, nhi, ntape, fuel-1), extpath)]
                
    return (nl_id_path, haltList)

 
       

In [8]:
step_tm(wpw, ("q0",0,"0#1", 3), [], set({}))

In [9]:
step_tm(wwndtm, ("q14",0, "00", 100), [], set({}))

# Routines to run TM

We now devise a routine to run an NDTM.

In [10]:
def run_tm(T, tape, fuel):
    """Given a TM T and a tape, run the TM for fuel steps
       (e.g., gallons of gas in your tank), 
       collecting all halting configurations. 
       
       Return a triple
       (id_path_pairs, haltSet, nfuel) of
        - all resulting id_path_pairs 
        - the final haltSet
        - the remaining fuel
       This way, one can find all accepting and rejecting 
       IDs in the final haltSet and print paths to them.
    """
    q_hi_tape_fuel = (T["q0"], 0, tape, fuel)
    path         = []
    l_id_path    = [ (q_hi_tape_fuel, path) ]
    haltList     = [ ]
    l_trunc_path = [] # List of truncated paths
    
    while (l_id_path != []):
        (q_hi_tape_fuel, path)  = l_id_path[0] 
        (nq, nhi, ntape, nfuel) = q_hi_tape_fuel
        if (nfuel > 0):
            (nl_id_path, haltList) = step_tm(T, q_hi_tape_fuel, 
                                             path, haltList)
            l_id_path = nl_id_path + l_id_path[1:]
        else:
            l_trunc_path += [path]
            l_id_path = l_id_path[1:]
    
    return (l_trunc_path, haltList)


In [11]:
def explore_tm(T, tape, nsteps):
    """A handy routine to print the result of run_tm plus making 
       future extensions to explore run-results.
    """
    (l_trunc_path, haltList) = run_tm(T, tape, nsteps)
    if (haltList == []):
        print("TM hasn't halted.")
        print("The truncated paths so far are as follows.")
        for trunc_path in l_trunc_path:
            print(trunc_path)
    else:
        if (l_trunc_path != []):
            print("There are still ", len(l_trunc_path), "truncated paths.")
        print("Detailing the halted configs now.")
        
        for (haltConfig, path) in haltList:
            (haltState, head, tape, fuel) = haltConfig
            if (haltState in T["F"]):
                print("Accepted at ", haltConfig)
            else:
                print("Rejected at ", haltConfig)
            print(" via .. ")
            for id in path:
                print(" ->", end="")
                print(id)
            print(" ->", end="")
            print(haltConfig)
   

In [12]:
run_tm(wpw, "01#01..", 19)

In [13]:
explore_tm(wpw, "01#01..", 19)

In [14]:
run_tm(wwndtm, "0100101001", 888)

In [15]:
explore_tm(wwndtm, "0100101001", 888)

In [16]:
explore_tm(wpw, "010#010", 44)

In [17]:
explore_tm(wpw, "010#011", 33)

In [18]:
explore_tm(wpw, "1#1", 54)

In [19]:
explore_tm(wpw, "01#01", 18)
#run_tm(wpw, "01#01", 18)

In [20]:
explore_tm(wpw, "010001101#010001101", 300)

In [21]:
wwndtm

In [22]:
dotObj_tm(wwndtm)

In [23]:
wwndtmobj = dotObj_tm(wwndtm)

In [24]:
wwndtmobj.source

In [25]:
explore_tm(wwndtm, "010010", 88)

In [26]:
explore_tm(wwndtm, "0101", 45)

In [27]:
explore_tm(wwndtm, "0100101001", 666)

In [28]:
explore_tm(wwndtm, "0100101001", 666)

In [29]:
explore_tm(wwndtm, "0100101001", 66)

In [30]:
addtm = {
    # This TM adds two numbers in base 2.
    #
    # Input: matches the regex "[01][01]*\+[01][01]*"
    #        Interpreted as a+b where a and b are unsigned integers in big-endian
    #          form
    # Output: Occurs on a halt in the "done" state.
    #         The sum of the numbers in big endian form.
    #         No other characters will be on the tape.
    #         There willbe no leading zeros on the answer.
    # Errors: The only errors that can occur are input errors which lead to
    #           a halt on the "error" state. This indicates the input does
    #           not conform to the input requirement.
    # Detail: There is no length maximums on the input.
    #         The numbers can be of different lengths.
    #         Leading zeros on inputs only cause a longer runtime.
    "Q": {
        # final states
        "done", "error",

        # input validation states
        "start",
        "check_a",
        "check_b_start",
        "check_b",

        # adder states
        "read_next_b_digit",
        "0_scan_to_a",
        "1_scan_to_a",
        "n_scan_to_a",
        "0_scan_to_a_digit",
        "1_scan_to_a_digit",
        "n_scan_to_a_digit",
        "0_scan_to_output",
        "1_scan_to_output",
        "c0_scan_to_output",
        "write_carry",
        "scan_to_b",
        "scan_to_b_digit",

        # answer formatting states
        "erase_until_sum",
        "find_end_of_sum",
        "read_msb_of_sum",
        "0_check_end_of_sum",
        "1_check_end_of_sum",
        "0_write_last_digit",
        "1_write_last_digit",
        "write_last_0",
        "write_last_1",
        "0_find_end_of_sum",
        "1_find_end_of_sum",
        "0_find_answer",
        "1_find_answer",
        "write_0",
        "write_1",
        "find_start_of_sum",
        "find_start_of_answer",
        "erase_leading_zeros",
    },

    "Sigma" : {'0', '1', '+'},

    "Gamma" : {'.',
               '0', '1', '+',
               'a', # right side marker for a input
               'b', # right side marker for b input
               'c', # carry indicator
               'X', # 0 used indicator
               'Y', # 1 used indicator
               's', # left side of answer space marker
               },

    "Delta" : {
        # Begin input validation
        ("start", '.') : { ("error", '.', 'S') },
        ("start", '+') : { ("error", '.', 'S') },
        ("start", '0') : { ("check_a", '0', 'R') },
        ("start", '1') : { ("check_a", '1', 'R') },
        #
        ("check_a", '0') : { ("check_a", '0', 'R') },
        ("check_a", '1') : { ("check_a", '1', 'R') },
        ("check_a", '.') : { ("error", '.', 'S') },
        ("check_a", '+') : { ("check_b_start", 'a', 'R') },
        #
        ("check_b_start", '0') : { ("check_b", '0', 'R') },
        ("check_b_start", '1') : { ("check_b", '1', 'R') },
        ("check_b_start", '.') : { ("error", '.', 'S') },
        ("check_b_start", '+') : { ("error", '+', 'S') },
        #
        ("check_b", '0') : { ("check_b", '0', 'R') },
        ("check_b", '1') : { ("check_b", '1', 'R') },
        ("check_b", '.') : { ("read_next_b_digit", 'b', 'L') },
        ("check_b", '+') : { ("error", '+', 'S') },
        # End input validation

        # Begin adder
        ("read_next_b_digit", '0') : { ("0_scan_to_a", 'X', 'L') },
        ("read_next_b_digit", '1') : { ("1_scan_to_a", 'Y', 'L') },
        #
        ("0_scan_to_a", '0') : { ("0_scan_to_a", '0', 'L') },
        ("0_scan_to_a", '1') : { ("0_scan_to_a", '1', 'L') },
        ("0_scan_to_a", 'a') : { ("0_scan_to_a_digit", 'a', 'L') },
        #
        ("1_scan_to_a", '0') : { ("1_scan_to_a", '0', 'L') },
        ("1_scan_to_a", '1') : { ("1_scan_to_a", '1', 'L') },
        ("1_scan_to_a", 'a') : { ("1_scan_to_a_digit", 'a', 'L') },
        #
        ("n_scan_to_a", '0') : { ("n_scan_to_a", '0', 'L') },
        ("n_scan_to_a", '1') : { ("n_scan_to_a", '1', 'L') },
        ("n_scan_to_a", 'a') : { ("n_scan_to_a_digit", 'a', 'L') },
        #
        ("0_scan_to_a_digit", 'X') : { ("0_scan_to_a_digit", 'X', 'L') },
        ("0_scan_to_a_digit", 'Y') : { ("0_scan_to_a_digit", 'Y', 'L') },
        ("0_scan_to_a_digit", '0') : { ("0_scan_to_output", 'X', 'R') },
        ("0_scan_to_a_digit", '1') : { ("1_scan_to_output", 'Y', 'R') },
        ("0_scan_to_a_digit", '.') : { ("0_scan_to_output", 's', 'R') },
        ("0_scan_to_a_digit", 's') : { ("0_scan_to_output", 's', 'R') },
        #
        ("1_scan_to_a_digit", 'X') : { ("1_scan_to_a_digit", 'X', 'L') },
        ("1_scan_to_a_digit", 'Y') : { ("1_scan_to_a_digit", 'Y', 'L') },
        ("1_scan_to_a_digit", '0') : { ("1_scan_to_output", 'X', 'R') },
        ("1_scan_to_a_digit", '1') : { ("c0_scan_to_output", 'Y', 'R') },
        ("1_scan_to_a_digit", '.') : { ("1_scan_to_output", 's', 'R') },
        ("1_scan_to_a_digit", 's') : { ("1_scan_to_output", 's', 'R') },
        #
        ("n_scan_to_a_digit", 'X') : { ("n_scan_to_a_digit", 'X', 'L') },
        ("n_scan_to_a_digit", 'Y') : { ("n_scan_to_a_digit", 'Y', 'L') },
        ("n_scan_to_a_digit", '0') : { ("0_scan_to_output", 'X', 'R') },
        ("n_scan_to_a_digit", '1') : { ("1_scan_to_output", 'Y', 'R') },
        ("n_scan_to_a_digit", '.') : { ("erase_until_sum", 's', 'R') },
        ("n_scan_to_a_digit", 's') : { ("erase_until_sum", 's', 'R') },
        #
        ("0_scan_to_output", '0') : { ("0_scan_to_output", '0', 'R') },
        ("0_scan_to_output", '1') : { ("0_scan_to_output", '1', 'R') },
        ("0_scan_to_output", 'X') : { ("0_scan_to_output", 'X', 'R') },
        ("0_scan_to_output", 'Y') : { ("0_scan_to_output", 'Y', 'R') },
        ("0_scan_to_output", 'a') : { ("0_scan_to_output", 'a', 'R') },
        ("0_scan_to_output", 'b') : { ("0_scan_to_output", 'b', 'R') },
        ("0_scan_to_output", '.') : { ("scan_to_b", '0', 'L') },
        ("0_scan_to_output", 'c') : { ("scan_to_b", '1', 'L') },
        #
        ("1_scan_to_output", '0') : { ("1_scan_to_output", '0', 'R') },
        ("1_scan_to_output", '1') : { ("1_scan_to_output", '1', 'R') },
        ("1_scan_to_output", 'X') : { ("1_scan_to_output", 'X', 'R') },
        ("1_scan_to_output", 'Y') : { ("1_scan_to_output", 'Y', 'R') },
        ("1_scan_to_output", 'a') : { ("1_scan_to_output", 'a', 'R') },
        ("1_scan_to_output", 'b') : { ("1_scan_to_output", 'b', 'R') },
        ("1_scan_to_output", '.') : { ("scan_to_b", '1', 'L') },
        ("1_scan_to_output", 'c') : { ("write_carry", '0', 'R') },
        #
        ("c0_scan_to_output", '0') : { ("c0_scan_to_output", '0', 'R') },
        ("c0_scan_to_output", '1') : { ("c0_scan_to_output", '1', 'R') },
        ("c0_scan_to_output", 'X') : { ("c0_scan_to_output", 'X', 'R') },
        ("c0_scan_to_output", 'Y') : { ("c0_scan_to_output", 'Y', 'R') },
        ("c0_scan_to_output", 'a') : { ("c0_scan_to_output", 'a', 'R') },
        ("c0_scan_to_output", 'b') : { ("c0_scan_to_output", 'b', 'R') },
        ("c0_scan_to_output", '.') : { ("write_carry", '0', 'R') },
        ("c0_scan_to_output", 'c') : { ("write_carry", '1', 'R') },
        #
        ("write_carry", '.') : { ("scan_to_b", 'c', 'L') },
        #
        ("scan_to_b", '0') : { ("scan_to_b", '0', 'L') },
        ("scan_to_b", '1') : { ("scan_to_b", '1', 'L') },
        ("scan_to_b", 'b') : { ("scan_to_b_digit", 'b', 'L') },
        #
        ("scan_to_b_digit", 'X') : { ("scan_to_b_digit", 'X', 'L') },
        ("scan_to_b_digit", 'Y') : { ("scan_to_b_digit", 'Y', 'L') },
        ("scan_to_b_digit", '0') : { ("read_next_b_digit", '0', 'S') },
        ("scan_to_b_digit", '1') : { ("read_next_b_digit", '1', 'S') },
        ("scan_to_b_digit", 'a') : { ("n_scan_to_a", 'a', 'S') },
        # End adder

        # Begin answer formatting
        ("erase_until_sum", '0') : { ("erase_until_sum", '.', 'R') },
        ("erase_until_sum", '1') : { ("erase_until_sum", '.', 'R') },
        ("erase_until_sum", 'X') : { ("erase_until_sum", '.', 'R') },
        ("erase_until_sum", 'Y') : { ("erase_until_sum", '.', 'R') },
        ("erase_until_sum", 'a') : { ("erase_until_sum", '.', 'R') },
        ("erase_until_sum", 'b') : { ("find_end_of_sum", '.', 'R') },
        #
        ("find_end_of_sum", '0') : { ("find_end_of_sum", '0', 'R') },
        ("find_end_of_sum", '1') : { ("find_end_of_sum", '1', 'R') },
        ("find_end_of_sum", 'c') : { ("read_msb_of_sum", '1', 'S') },
        ("find_end_of_sum", '.') : { ("read_msb_of_sum", '.', 'L') },
        #
        ("read_msb_of_sum", '0') : { ("0_check_end_of_sum", '.', 'L') },
        ("read_msb_of_sum", '1') : { ("1_check_end_of_sum", '.', 'L') },
        #
        ("0_check_end_of_sum", '0') : { ("0_find_end_of_sum", '0', 'L') },
        ("0_check_end_of_sum", '1') : { ("0_find_end_of_sum", '1', 'L') },
        ("0_check_end_of_sum", '.') : { ("0_write_last_digit", '.', 'L') },
        #
        ("1_check_end_of_sum", '0') : { ("1_find_end_of_sum", '0', 'L') },
        ("1_check_end_of_sum", '1') : { ("1_find_end_of_sum", '1', 'L') },
        ("1_check_end_of_sum", '.') : { ("1_write_last_digit", '.', 'L') },
        #
        ("0_write_last_digit", '.') : { ("0_write_last_digit", '.', 'L') },
        ("0_write_last_digit", '0') : { ("write_last_0", '0', 'R') },
        ("0_write_last_digit", '1') : { ("write_last_0", '1', 'R') },
        ("0_write_last_digit", 's') : { ("write_last_0", '.', 'R') },
        #
        ("1_write_last_digit", '.') : { ("1_write_last_digit", '.', 'L') },
        ("1_write_last_digit", '0') : { ("write_last_1", '0', 'R') },
        ("1_write_last_digit", '1') : { ("write_last_1", '1', 'R') },
        ("1_write_last_digit", 's') : { ("write_last_1", '.', 'R') },
        #
        ("write_last_0", '.') : { ("find_start_of_answer", '0', 'S') },
        #
        ("write_last_1", '.') : { ("find_start_of_answer", '1', 'S') },
        #
        ("0_find_end_of_sum", '0') : { ("0_find_end_of_sum", '0', 'L') },
        ("0_find_end_of_sum", '1') : { ("0_find_end_of_sum", '1', 'L') },
        ("0_find_end_of_sum", '.') : { ("0_find_answer", '.', 'L') },
        #
        ("1_find_end_of_sum", '0') : { ("1_find_end_of_sum", '0', 'L') },
        ("1_find_end_of_sum", '1') : { ("1_find_end_of_sum", '1', 'L') },
        ("1_find_end_of_sum", '.') : { ("1_find_answer", '.', 'L') },
        #
        ("0_find_answer", '.') : { ("0_find_answer", '.', 'L') },
        ("0_find_answer", '0') : { ("write_0", '0', 'R') },
        ("0_find_answer", '1') : { ("write_0", '1', 'R') },
        ("0_find_answer", 's') : { ("write_0", '.', 'R') },
        #
        ("1_find_answer", '.') : { ("1_find_answer", '.', 'L') },
        ("1_find_answer", '0') : { ("write_1", '0', 'R') },
        ("1_find_answer", '1') : { ("write_1", '1', 'R') },
        ("1_find_answer", 's') : { ("write_1", '.', 'R') },
        #
        ("write_0", '.') : { ("find_start_of_sum", '0', 'R') },
        #
        ("write_1", '.') : { ("find_start_of_sum", '1', 'R') },
        #
        ("find_start_of_sum", '.') : { ("find_start_of_sum", '.', 'R') },
        ("find_start_of_sum", '0') : { ("find_end_of_sum", '0', 'S') },
        ("find_start_of_sum", '1') : { ("find_end_of_sum", '1', 'S') },
        #
        ("find_start_of_answer", '0') : { ("find_start_of_answer", '0', 'L') },
        ("find_start_of_answer", '1') : { ("find_start_of_answer", '1', 'L') },
        ("find_start_of_answer", '.') : { ("erase_leading_zeros", '.', 'R') },
        #
        ("erase_leading_zeros", '0') : { ("erase_leading_zeros", '.', 'R') },
        ("erase_leading_zeros", '1') : { ("done", '1', 'S') },
        ("erase_leading_zeros", '.') : { ("done", '0', 'S') },
        # End answer formatting
    },

    "q0"    : "start",

    "B"     : '.',

    "F"     : {"done"}
}

In [31]:
dotObj_tm(addtm)

In [32]:
addtmobj = dotObj_tm(addtm)

In [33]:
addtmobj.source

In [34]:
explore_tm(addtm, "11111101+11111101", 735)